## Arsitektur NN

In [2]:
from jcopdl.layers import linear_block, conv_block
import torch
from torch import nn, optim
from jcopdl.callback import Callback, set_config

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
class CifarCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            conv_block(3, 16), # 3x32x32 -> 16x16x16
            conv_block(16, 32), # 16x16x16 -> 32x8x8 
            conv_block(32, 64), #32x8x8 -> 64x4x4
            nn.Flatten()
        )
        
        self.fc = nn.Sequential(
            linear_block(1024, 512, dropout=0.1),
            linear_block(512, 64, dropout=0.1),
            linear_block(64, 10, activation="lsoftmax")
        )
        
    def forward(self, x):
        return self.fc(self.conv(x))

In [ ]:
config = set_config({
    "bs" : 123,
    "crop_size" : 32,
    "input_layer" : 1024,
    "hidden1" : 512,
    "hidden2" : 64,
    "output_size" : len(label2cat),
    "drout" : 0.1
})

In [ ]:
from tqdm.auto import tqdm

def loop_fn(mode, dataset, dataloader, model, criterion, optimizer, device):
    if mode == "train":
        model.train()
    elif mode == "test":
        model.eval()
    cost = correct = 0
    for feature, target in tqdm(dataloader, desc=mode.title()):
        feature, target = feature.to(device), target.to(device)
        output = model(feature)
        loss = criterion(output, target)
        
        if mode == "train":
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        
        cost += loss.item() * feature.shape[0]
        correct += (output.argmax(1) == target).sum().item()
    cost = cost / len(dataset)
    acc = correct / len(dataset)
    return cost, acc

In [ ]:
while True:
    train_cost, train_score = loop_fn("train", train_set, trainloader, model, criterion, optimizer, device)
    with torch.no_grad():
        test_cost, test_score = loop_fn("test", test_set, testloader, model, criterion, optimizer, device)
    
    # Logging
    callback.log(train_cost, test_cost, train_score, test_score)

    # Checkpoint
    callback.save_checkpoint()
        
    # Runtime Plotting
    callback.cost_runtime_plotting()
    callback.score_runtime_plotting()
    
    # Early Stopping
    if callback.early_stopping(model, monitor="test_score"):
        callback.plot_cost()
        callback.plot_score()
        break